In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
files = ['2011-q1_trip_history_data.csv','2011-q2_trip_history_data.csv','2011-q3_trip_history_data.csv','2011-q4_trip_history_data.csv']

In [3]:
list_data = []
df_2011 = pd.DataFrame()
for file in files:
    df_qdata = pd.read_csv(file)
    list_data.append(df_qdata)
df_2011 = pd.concat(list_data)

df_2011['StartTime'] = df_2011['Start date'].astype('datetime64')
df_2011['EndTime'] = df_2011['End date'].astype('datetime64')
df_2011.drop(['Start date','End date'],axis=1,inplace=True)

In [3]:
df_2011.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226768 entries, 0 to 309383
Data columns (total 9 columns):
Duration (ms)           1226768 non-null int64
Start station number    1226768 non-null int64
Start station           1226768 non-null object
End station number      1226768 non-null int64
End station             1226768 non-null object
Bike number             1226768 non-null object
Member type             1226768 non-null object
StartTime               1226768 non-null datetime64[ns]
EndTime                 1226768 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 93.6+ MB


In [4]:
group_bikes = df_2011.groupby('Bike number')
group_origin = df_2011.groupby('Start station number')
group_dest = df_2011.groupby('End station number')

In [5]:
df_2011.groupby(['Start station number',pd.TimeGrouper('D',key='StartTime')])['Start station'].count().sort_values(ascending=False)

Start station number  StartTime 
31200                 2011-06-04    275
                      2011-07-16    275
                      2011-09-10    268
                      2011-10-09    258
                      2011-06-29    248
                      2011-06-05    247
                      2011-04-30    246
                      2011-09-25    241
                      2011-09-24    241
                      2011-05-29    240
                      2011-07-17    239
                      2011-07-14    235
                      2011-07-31    235
                      2011-06-26    233
                      2011-09-04    229
                      2011-08-20    229
                      2011-08-26    228
                      2011-05-21    225
                      2011-10-16    225
                      2011-10-15    225
                      2011-08-30    224
                      2011-06-28    224
                      2011-06-13    222
                      2011-06-03    221
       

In [9]:
df_addresses = df_2011.groupby('Start station number')['Start station'].last()
df_addresses.to_csv('stationlocation.csv')

In [7]:
all_bike_journeys = []
for name,group in group_bikes:
    df_biketrip = pd.DataFrame()
    df_biketrip['BikeNum'] = group['Bike number']
    df_biketrip['MemType'] = group['Member type']
    df_biketrip['StartNum'] = group['Start station number'].astype(float)
    df_biketrip['EndNum'] = group['End station number'].astype(float)
    df_biketrip['LastEnd'] = group.shift(1)['End station number']
    df_biketrip['StartTime'] = group['StartTime']
    df_biketrip['EndTime'] = group['EndTime']
    df_biketrip['LastEndTime'] = group.shift(1)['EndTime']
    df_biketrip['count'] = 1
    df_biketrip['TripDuration'] = group['Duration (ms)']/1000
    df_biketrip['StartIsLastEnd'] = (df_biketrip['LastEnd'] == df_biketrip['StartNum']).astype(int)
    df_biketrip['DurationSinceLastTrip'] = np.where(df_biketrip['StartIsLastEnd'],(((df_biketrip['StartTime']-df_biketrip['LastEndTime']).fillna(0))/np.timedelta64(1,'s')).astype(int),-1)

    df_biketrip['bikeavailunder1m'] = (df_biketrip['DurationSinceLastTrip'] < 60).astype(int)
    df_biketrip['bikeavailunder10m'] = (df_biketrip['DurationSinceLastTrip'] < 600).astype(int)
    df_biketrip['bikeavailunder1h'] = (df_biketrip['DurationSinceLastTrip'] < 3600).astype(int)
    df_biketrip['bikeavailover12h'] = (df_biketrip['DurationSinceLastTrip'] > 43200).astype(int)
    df_biketrip['bikeavailover1d'] = (df_biketrip['DurationSinceLastTrip'] > 86400).astype(int)
    
    df_biketrip['biketripunder1m'] = (df_biketrip['TripDuration'] < 60).astype(int)
    df_biketrip['biketripunder10m'] = (df_biketrip['TripDuration'] < 600).astype(int)
    df_biketrip['biketripunder1h'] = (df_biketrip['TripDuration'] < 3600).astype(int)
    df_biketrip['biketripover1h'] = (df_biketrip['TripDuration'] > 3600).astype(int)
    df_biketrip['biketripover1d'] = (df_biketrip['TripDuration'] > 86400).astype(int)
    
    all_bike_journeys.append(df_biketrip)
df_all_biketrips = pd.concat(all_bike_journeys)

In [8]:
df_all_biketrips.to_csv('biketrips2011.csv')

In [11]:
#to get thoroughput, look at # of end trips per hour per site and # of start trips per hour per site
#a positive number means bikes are coming in
#a negative number means bikes are being drawn down
df_hourly_dest_count = pd.DataFrame()
df_hourly_origin_count = pd.DataFrame()
for name,group in group_dest:
    df_hourly_dest_count[name] = group.groupby(pd.TimeGrouper('H',key='StartTime'))['End station number'].count()
for name,group in group_origin:
    df_hourly_origin_count[name] = group.groupby(pd.TimeGrouper('H',key='StartTime'))['Start station number'].count()
df_thoroughput_hourly = df_hourly_dest_count-df_hourly_origin_count
df_thoroughput_hourly.to_csv('thoroughput2011hourly.csv')


None


In [12]:
df_daily_dest_count = pd.DataFrame()
df_daily_origin_count = pd.DataFrame()
for name,group in group_dest:
    df_daily_dest_count[name] = group.groupby(pd.TimeGrouper('D',key='StartTime'))['End station number'].count()
for name,group in group_origin:
    df_daily_origin_count[name] = group.groupby(pd.TimeGrouper('D',key='StartTime'))['Start station number'].count()
df_thoroughput_daily = df_daily_dest_count-df_daily_origin_count
df_thoroughput_daily.to_csv('thoroughput2011daily.csv')